# Redes Siamesas 

En este cuaderno vamos a construir una red siamesa

Una novedad importante de este cuaderno es que vamos a utilizar una capa con una función que diseñaremos nosotros.

---

    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es 

In [1]:
#-[0]. General purpose packages

import numpy as np
import matplotlib.pyplot as plt
np.random.seed(2020) ##<- for reproducibility

#-[1]. Load images. Keras has a few benchmark datasets readily available.

from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#--- Get info of train and test data sets
N_train,dim0,dim1 = X_train.shape
N_test,dim0,dim1  = X_test.shape
input_dim = dim0*dim1

# the data, shuffled and split between train and test sets
X_train = X_train.reshape(N_train, input_dim)
X_test = X_test.reshape(N_test, input_dim)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

## Creación de la red

In [2]:
#from __future__ import absolute_import
#from __future__ import print_function

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, Lambda
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras import backend as K

#### Distancia L1 punto a punto entre dos vectores

In [3]:
def get_abs_diff(vects):
    # L1 distance between two vectors
    x, y = vects
    return K.abs(x - y)

#### Función para devolver las dims. de la "capa" que calcula la dif. L1.

In [4]:
def abs_diff_output_shape(shapes):
    shape1, shape2 = shapes
    return shape1 

#### Crear la red neuronal *base*

In [5]:
def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq

#### Crear la red siamesa y compilar el modelo

In [6]:
# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)


abs_diff = Lambda(get_abs_diff, output_shape = abs_diff_output_shape)([processed_a, processed_b])

flattened_weighted_distance = Dense(1, activation = 'sigmoid')(abs_diff)

model = Model([input_a, input_b], flattened_weighted_distance)

rms = RMSprop()
model.compile(loss = 'binary_crossentropy', optimizer=rms, metrics = ['accuracy'])

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 128)          133504      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 128)          0           sequential[0][0]      

## Entrenamiento

#### Función para crear Pares de ejemplos positivos y negativos a partir del conjunto de entrenamiento

In [7]:
import random
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

### Crear conjuntos etiquetados de entrenamiento y test

In [8]:
# create training+test positive and negative pairs

digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(X_test, digit_indices)

### Entrenar

In [9]:
# train

epochs = 5

model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          batch_size=128, epochs=epochs)

Epoch 1/5
847/847 [==============================] - 4s 5ms/step - loss: 0.3156 - accuracy: 0.8640 - val_loss: 0.2473 - val_accuracy: 0.9152
Epoch 2/5
847/847 [==============================] - 4s 5ms/step - loss: 0.1533 - accuracy: 0.9439 - val_loss: 0.1508 - val_accuracy: 0.9519
Epoch 3/5
847/847 [==============================] - 4s 5ms/step - loss: 0.1006 - accuracy: 0.9641 - val_loss: 0.1125 - val_accuracy: 0.9664
Epoch 4/5
847/847 [==============================] - 4s 5ms/step - loss: 0.0764 - accuracy: 0.9731 - val_loss: 0.0864 - val_accuracy: 0.9726
Epoch 5/5
847/847 [==============================] - 4s 5ms/step - loss: 0.0623 - accuracy: 0.9781 - val_loss: 0.0881 - val_accuracy: 0.9730


### Test

In [10]:
from sklearn.metrics import accuracy_score as accuracy

# compute final accuracy on training and test sets
tr_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = accuracy(tr_y, tr_pred.round())

te_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = accuracy(te_y, te_pred.round())

print('* Accuracy on the training set: {:.2%}'.format(tr_acc))
print('* Accuracy on the test set: {:.2%}'.format(te_acc))



* Accuracy on the training set: 98.52%
* Accuracy on the test set: 97.30%
